In [105]:
from typing import Dict,Text
import pandas as pd
import tensorflow as tf
import tensorflow_recommenders as tfrs
import numpy as np
import duckdb
import json
con = duckdb.connect('database.db')

#PREPARE DATA

podcast_ratings_query = con.sql("select author_id as user_id, podcasts.title as podcast_title,rating from reviews "+ 
        "join categories using (podcast_id) " +
        "join podcasts using (podcast_id) where average_rating >= 0").to_df()

podcast_titles_query = con.sql("select podcasts.title as podcast_title from reviews "+ 
        "join categories using (podcast_id) " +
        "join podcasts using (podcast_id) where average_rating >= 0").to_df()

podcast_ratings = podcast_ratings_query.to_dict(orient='records')
podcast_titles = podcast_titles_query.to_dict(orient='records')
podcast_ratings_tf = tf.data.Dataset.from_tensor_slices(pd.DataFrame.from_dict(podcast_ratings).to_dict(orient="list"))
podcast_titles_tf = tf.data.Dataset.from_tensor_slices(pd.DataFrame.from_dict(podcast_titles).to_dict(orient="list"))

ratings_tf = podcast_ratings_tf.map(lambda x: {
    "podcast_title": x["podcast_title"],
    "user_id": x["user_id"],
    "user_rating": x["rating"]
})
titles_tf = podcast_titles_tf.map(lambda x: x["podcast_title"])

tf.random.set_seed(42)
shuffled = ratings_tf.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train_len = int(len(shuffled) * (2/3))
test_len = int(len(shuffled) - train_len)

train = shuffled.take(train_len)
test = shuffled.skip(train_len).take(test_len)

podcast_titles = titles_tf.batch(1_000)
user_ids = ratings_tf.batch(1_000_000).map(lambda x: x["user_id"])

unique_podcast_titles = np.unique(np.concatenate(list(podcast_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

In [106]:
#TRAIN MODEL

class PodcastlensModel(tfrs.Model):

  def __init__(self):
    super().__init__()
    embedding_dimension = 32
    self.user_model = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
          vocabulary=unique_user_ids, mask_token=None),
      # We add an additional embedding to account for unknown tokens.
      tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
    ])

    self.podcast_model = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
          vocabulary=unique_podcast_titles, mask_token=None),
      tf.keras.layers.Embedding(len(unique_podcast_titles) + 1, embedding_dimension)
    ])
    metrics = tfrs.metrics.FactorizedTopK(
      candidates=titles_tf.batch(128).map(self.podcast_model)
    )

    self.task = tfrs.tasks.Retrieval(
      metrics=metrics
    )

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model(features["user_id"])
    # And pick out the podcast features and pass them into the podcast model,
    # getting embeddings back.
    positive_podcast_embeddings = self.podcast_model(features["podcast_title"])

    # The task computes the loss and the metrics.
    return self.task(user_embeddings, positive_podcast_embeddings)

model = PodcastlensModel()
model.compile(optimizer=tf.keras.optimizers.Adamax(learning_rate=0.1))
cached_train = train.shuffle(100_000).batch(250).cache()
cached_test = test.batch(100).cache()
model.fit(cached_train, epochs=25)

# Create a model that takes in raw query features, and
index = tfrs.layers.factorized_top_k.Streaming(model.user_model,k=75)
# recommends podcasts out of the entire podcasts dataset.
index.index_from_dataset(
  tf.data.Dataset.zip((titles_tf.batch(100), titles_tf.batch(100).map(model.podcast_model)))
)

Epoch 1/25
3/3 [==============================] - 2s 88ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0019 - factorized_top_k/top_50_categorical_accuracy: 0.0448 - factorized_top_k/top_100_categorical_accuracy: 0.0994 - loss: 700.6003 - regularization_loss: 0.0000e+00 - total_loss: 700.6003 
Epoch 2/25
3/3 [==============================] - 0s 90ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0175 - factorized_top_k/top_10_categorical_accuracy: 0.0487 - factorized_top_k/top_50_categorical_accuracy: 0.2203 - factorized_top_k/top_100_categorical_accuracy: 0.4542 - loss: 617.3085 - regularization_loss: 0.0000e+00 - total_loss: 617.3085
Epoch 3/25
3/3 [==============================] - 0s 86ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accurac

In [107]:
#PULL METRICS FOR MODEL ON TEST AND TRAIN DATA
train_metrics = model.evaluate(cached_train, return_dict=True)
test_metrics = model.evaluate(cached_test, return_dict=True)
print("Train Metrics")
print(json.dumps(train_metrics,indent=4))
print("") 
print("Test Metrics")
print(json.dumps(test_metrics,indent=4))

3/3 [==============================] - 0s 87ms/step - factorized_top_k/top_1_categorical_accuracy: 0.6381 - factorized_top_k/top_5_categorical_accuracy: 0.6654 - factorized_top_k/top_10_categorical_accuracy: 0.6654 - factorized_top_k/top_50_categorical_accuracy: 0.7315 - factorized_top_k/top_100_categorical_accuracy: 0.7665 - loss: 203.3676 - regularization_loss: 0.0000e+00 - total_loss: 203.3676
Train Metrics
{
    "factorized_top_k/top_1_categorical_accuracy": 0.7115010023117065,
    "factorized_top_k/top_5_categorical_accuracy": 0.7446393966674805,
    "factorized_top_k/top_10_categorical_accuracy": 0.7504873275756836,
    "factorized_top_k/top_50_categorical_accuracy": 0.8167641162872314,
    "factorized_top_k/top_100_categorical_accuracy": 0.8654970526695251,
    "loss": 8.841540336608887,
    "regularization_loss": 0,
    "total_loss": 8.841540336608887
}

Test Metrics
{
    "factorized_top_k/top_1_categorical_accuracy": 0.6381322741508484,
    "factorized_top_k/top_5_categorical

In [108]:
#RUN PREDICTIONS ON TEST DATA 
num = 0 
threshold = 1
for element in test:
    user_id = element["user_id"].numpy().decode()
    _, titles = index(tf.constant([user_id]))
    unique_preds = np.unique(titles.numpy())
    unique_preds = [el.decode('UTF-8') for el in unique_preds]
    if len(unique_preds) > threshold:
        print("Top {} recommendations for user {}: {}".format(len(unique_preds),user_id,unique_preds))
        print("")
        num += 1
print("Number of users with more than {} recommendation: {}".format(threshold,num))

Top 9 recommendations for user a4468d88bd0980f: ['Daily Solutions Podcast', 'Emprendedores Digitales |Marketing Digital, Blogging, Redes Sociales, Marketing Online, Negocios, SEO, blogs, Desarrollo Pers', 'Last Week at the Movies Podcast', 'Maximiza Tu Negocio en Redes de Mercadeo', 'NFL no ProFootballcast com Antony Curti e Eduardo Miceli', 'Rebel Chums', 'Talking Web Marketing', 'The Fertility Warriors Podcast: Helping women survive infertility and trying to conceive', 'Twin Talk with the King Twins']

Top 2 recommendations for user 0886b9032023e92: ['Rekt Podcast', 'Twin Talk with the King Twins']

Top 9 recommendations for user 353d234db877315: ['All of the Above radio', 'Daily Solutions Podcast', 'In Real Life with Emily and Kimzilla | WFMU', 'Lost In The Shuffle', 'New Patient Group™ (Formally known as the Doctor Diamond Club Podcast)', 'Tall Tale TV', 'Things a Teacher Taught Me', 'Twin Talk with the King Twins', 'Understanding Human Behavior - Video']

Top 6 recommendations for